<a href="https://colab.research.google.com/github/basse058/Big-Data-ETL/blob/main/part_one_grocery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [908 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-12 00:41:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.49MB/s    in 0.2s    

2023-03-12 00:41:43 (5.49 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz" #enter correct address here

spark.sparkContext.addFile(url)  #No need to use sparkcontext because we are reading it from a local folder

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")

df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...|2015-01-31 00:08:00|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...| 

In [6]:
# Get the number of rows in the DataFrame.
df.count()


2402458

# Transform the Data

## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = df.select(['review_id','customer_id','product_id','product_parent',to_date(df.review_date, 'yyyy-MM-dd').alias('review_date')])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740| 2015-01-31|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402| 2015-01-31|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703| 2015-01-31|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342| 2015-01-31|
| RTWHVNV6X4CNJ|   18123821|B004ZWR9RQ|     552138758| 2015-01-31|
| RIG9AWFOGRDVO|   23649464|B00AL6QBZ6|     681475449| 2015-01-31|
|R1S1XSG4ZCHDGS|   32778285|B00BCNSTRA|     578681693| 2015-01-31|
| RB15NBVY5ELVW|   46612941|B0089MM2BG|     350968436| 2015-01-31|
| R56358YM1ZJ7I|   31525399|B00Y1C9770|     729982780| 2015-01-31|
|R1ODXB3C9UP3NL|   19624355|B00J074W94|       2499702| 2015-01-31|
|R155ZMVLD5C9BP|   31910375|B0001VKKOO|     189938205| 2015-01-31|
|R2IXW43PUNYM0G|   12816533|B007TGH4CK|     398128262| 2015-01

## Create the "products" Table

In [8]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(['product_id','product_title'])
products_df = products_df.dropDuplicates(['product_id'])
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0657745316|100 Percent All N...|
|0681727810|Beemster Gouda - ...|
|0700026444|Pure Darjeeling T...|
|0786960159|Axis and Allies 1...|
|109274665X|Nestle Milkybar D...|
|1403796890|WWE Kids Todler V...|
|1453060464|Splash Energy Liq...|
|1601263724|      Speaking Amish|
|1603112251|Cocktail Kingdom ...|
|1625684479|Peter Rabbit &amp...|
|1837993823|HiPP Organic Comb...|
|1837993955|Milky Way Magic S...|
|2123457892|1 Month Suply (4 ...|
|2533790060|Ferrero Big Roche...|
|2635214875|Iaso Nutra Burst ...|
|3301261876|Haribo Jelly Babi...|
|3334353648|Fresh Ginger Root...|
|3355335541|Lipton Yellow Lab...|
|359595954X|Haldiram's Classi...|
|3621813667|HiPP Organic Firs...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [9]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import desc
customers_df = df.groupby('customer_id').agg({"customer_id":"count"}).withColumnRenamed('count(customer_id)', 'customer_cnt')
customers_df.orderBy(desc('customer_cnt')).show()


+-----------+------------+
|customer_id|customer_cnt|
+-----------+------------+
|   34247947|        1206|
|   17407953|         893|
|   12201275|         737|
|   52545115|         606|
|   53037408|         479|
|   50820654|         458|
|   50199793|         454|
|   37446839|         450|
|    7080939|         445|
|   51812418|         419|
|   50673475|         401|
|   15051518|         397|
|   26221913|         351|
|   39789300|         343|
|   28516018|         316|
|   52989182|         305|
|   44834233|         282|
|   14509190|         278|
|   52833354|         275|
|   52657160|         269|
+-----------+------------+
only showing top 20 rows



## Create the "vine_table".

In [10]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R26MV8D0KG6QI6|          5|            0|          0|   N|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|
| RIG9AWFOGRDVO|          2|            1|          1|   N|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|
| RB15NBVY5ELVW|          5|            2|          2|   N|
| R56358YM1ZJ7I|          5|            0|          0|   N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|
|R155ZMVLD5C9BP|          4|            3|          4|   N|
|R2IXW43PUNYM0G|          4|            1|          1|   N|
|R3LYEG1QCK2BG0|          5|            0|          0|   N|
|R20LREICPM3YH0|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://my-data-class-db.c4q3joupwllm.us-east-1.rds.amazonaws.com:5432/my-data-class-db"
config = {"user":"postgres", "password": "testpass", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)